## [단어 레벨 seq2seq](https://wikidocs.net/86900)

In [ ]:
import numpy as np
import re
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os
import unicodedata
import urllib3
import zipfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data = pd.read_excel('/content/drive/My Drive/seq2seq_3조/seq2seq_aihub/kor.xlsx',sheet_name='Sheet1')
src = pd.DataFrame(data['ko'])
tar = pd.DataFrame(data['en'])
df = pd.concat([src, tar],axis=1)
df

,ko,en
0,나는 매일 저녁 배트를 만나러 다락방으로 가요.,I go to the attic every evening to meet Bat.
1,선생님 이문장이 이해가 안 가요.,"Sir, I don't understand this sentence here."
2,컴퓨터를 시작하면 시간이 너무 빠르게 가요.,Time flies when you start using the computer.
3,나는 오늘 자정에 한국으로 돌아 가요.,I'm going back to Korea today at midnight.
4,나는 일어나자마자 화장실에 가요.,I go to bathroom as soon as I wake up.
...,...,...
74995,나의 고민은 학교가 멀어서 통학하기 힘들어.,My worry is commuting to school because it's t...
74996,난 지금 내고양이때문에 충분히 힘들어.,I am going under enough difficulties because o...
74997,나와 대화가 어려운 것이 많이 힘들어?,Is having difficulties in talking with me too ...
74998,하루에 한번 연락하는게 그렇게 힘들어?,Is it that difficult to call once a day?


### Preprocessing

In [ ]:
def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    #sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^ㄱ-ㅎㅏ|가-힣a-zA-Z!.?]+", r" ", sent)
   #sent = re.sub(r"[^ㄱ-ㅎㅏ|가-힣]+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [ ]:
# 전처리 테스트
# ko_sent = u"너 저녁 먹었어?"
# en_sent = u"Have you had dinner?"
print(preprocess_sentence(df.iloc[0][0]))
print(preprocess_sentence(df.iloc[0][1]))

나는 매일 저녁 배트를 만나러 다락방으로 가요 .
I go to the attic every evening to meet Bat .


In [ ]:
df.iloc[0][0].strip()

'나는 매일 저녁 배트를 만나러 다락방으로 가요.'

In [ ]:
[w for w in preprocess_sentence(df.iloc[0][0].strip()).split()]

['나는', '매일', '저녁', '배트를', '만나러', '다락방으로', '가요', '.']

In [ ]:
def load_preprocessed_data(data):
    encoder_input, decoder_input, decoder_target = [], [], []

    # source 데이터와 target 데이터 분리
    for i in range(len(data)):
        src_line = data.iloc[i][0].strip()
        tar_line = data.iloc[i][1].strip()

        # source 데이터 전처리
        src_line_input = [w for w in preprocess_sentence(src_line).split()]

        # target 데이터 전처리
        tar_line = preprocess_sentence(tar_line)
        tar_line_input = [w for w in ("<sos> " + tar_line).split()]
        tar_line_target = [w for w in (tar_line + " <eos>").split()]

        encoder_input.append(src_line_input[::-1])
        decoder_input.append(tar_line_input)
        decoder_target.append(tar_line_target)

    return encoder_input, decoder_input, decoder_target

In [ ]:
# 이렇게 얻은 3개의 데이터셋은 인코더의 입력, 디코더의 입력, 디코더의 실제값을 상위 5개 샘플만 출력
sents_ko_in, sents_en_in, sents_en_out = load_preprocessed_data(df)

In [ ]:
print(sents_ko_in[:5])
print(sents_en_in[:5])
print(sents_en_out[:5])

[['.', '가요', '다락방으로', '만나러', '배트를', '저녁', '매일', '나는'], ['.', '가요', '안', '이해가', '이문장이', '선생님'], ['.', '가요', '빠르게', '너무', '시간이', '시작하면', '컴퓨터를'], ['.', '가요', '돌아', '한국으로', '자정에', '오늘', '나는'], ['.', '가요', '화장실에', '일어나자마자', '나는']]
[['<sos>', 'I', 'go', 'to', 'the', 'attic', 'every', 'evening', 'to', 'meet', 'Bat', '.'], ['<sos>', 'Sir', 'I', 'don', 't', 'understand', 'this', 'sentence', 'here', '.'], ['<sos>', 'Time', 'flies', 'when', 'you', 'start', 'using', 'the', 'computer', '.'], ['<sos>', 'I', 'm', 'going', 'back', 'to', 'Korea', 'today', 'at', 'midnight', '.'], ['<sos>', 'I', 'go', 'to', 'bathroom', 'as', 'soon', 'as', 'I', 'wake', 'up', '.']]
[['I', 'go', 'to', 'the', 'attic', 'every', 'evening', 'to', 'meet', 'Bat', '.', '<eos>'], ['Sir', 'I', 'don', 't', 'understand', 'this', 'sentence', 'here', '.', '<eos>'], ['Time', 'flies', 'when', 'you', 'start', 'using', 'the', 'computer', '.', '<eos>'], ['I', 'm', 'going', 'back', 'to', 'Korea', 'today', 'at', 'midnight', '.', '<eos>'], ['I

In [ ]:
len(sents_ko_in)

75000

In [ ]:
tokenizer_ko = Tokenizer(filters="",lower=False)
tokenizer_ko.fit_on_texts(sents_ko_in)
encoder_input = tokenizer_ko.texts_to_sequences(sents_ko_in)

tokenizer_en = Tokenizer(filters="",lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
tokenizer_en.fit_on_texts(sents_en_out)
decoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
decoder_target = tokenizer_en.texts_to_sequences(sents_en_out)

In [ ]:
tokenizer_ko

In [ ]:
Tokenizer?

In [ ]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [ ]:
encoder_input.shape, decoder_input.shape, decoder_target.shape

((75000, 15), (75000, 19), (75000, 19))

In [ ]:
src_vocab_size = len(tokenizer_ko.word_index) + 1
tar_vocab_size = len(tokenizer_en.word_index) + 1
print("한국어 단어 집합의 크기 : {:d}, 영어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

한국어 단어 집합의 크기 : 85238, 영어 단어 집합의 크기 : 22349


In [ ]:
src_to_index = tokenizer_ko.word_index
index_to_src = tokenizer_ko.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_en.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

In [ ]:
encoder_input.shape[0]

75000

In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[28719 33300 48168 ...  1930 36279 51701]


In [ ]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [ ]:
encoder_input

array([[    1,     7,  3692, ...,     0,     0,     0],
       [    1,    24,    94, ...,     0,     0,     0],
       [    1,     5,   815, ...,     0,     0,     0],
       ...,
       [    1, 65822,  1154, ...,     0,     0,     0],
       [    2,   364,  3027, ...,     0,     0,     0],
       [    1,    73,  7259, ...,     0,     0,     0]], dtype=int32)

In [ ]:
# 이때, decoder_input과 decoder_target은 데이터의 구조상으로 앞에 붙은 <sos> 토큰과 뒤에 붙은 <eos>을 제외하면 동일한 정수 시퀀스를 가져야하므로 이를 확인
decoder_input[30997]

array([    2, 11655, 11656,  7218,     9,    63,  1048,     1,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0], dtype=int32)

In [ ]:
decoder_target[30997]

array([11655, 11656,  7218,     9,    63,  1048,     1,     3,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0], dtype=int32)

In [ ]:
# 훈련 데이터의 10%를 테스트 데이터로 분리
n_of_val = int(len(df)*0.1)
print(n_of_val)

7500


In [ ]:
# 10%에 해당되는 데이터를 테스트 데이터로 사용
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [ ]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(67500, 15)
(67500, 19)
(67500, 19)
(7500, 15)
(7500, 19)
(7500, 19)


## 모델 설계

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [ ]:
!python --version

Python 3.6.9


In [ ]:
# 임베딩 벡터와 LSTM의 은닉 상태의 크기를 특정 크기로 고정하고자 합니다.
latent_dim = 50

In [ ]:
# 인코더를 설계합니다. Masking은 패딩 토큰인 숫자 0의 경우에는 연산을 제외하는 역할을 수행
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [ ]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
decoder_lstm(dec_masking)

[<tf.Tensor 'lstm_1/PartitionedCall_1:1' shape=(None, None, 50) dtype=float32>,
 <tf.Tensor 'lstm_1/PartitionedCall_1:2' shape=(None, 50) dtype=float32>,
 <tf.Tensor 'lstm_1/PartitionedCall_1:3' shape=(None, 50) dtype=float32>]

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

**categorical_crossentropy를 사용하려면 레이블은 원-핫 인코딩이 된 상태여야 합니다. 그런데 현재 decoder_outputs의 경우에는 원-핫 인코딩을 하지 않은 상태입니다. 원-핫 인코딩을 하지 않은 상태로, 정수 레이블에 대해서 다중 클래스 분류 문제를 풀고자 하는 경우에는 categorical_crossentropy함수가 아니라 sparse_categorical_crossentropy를 사용**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import SGD

In [ ]:
# learning rate 조절하기 
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr/2*(epoch-5)
batch_size = 128
epochs = 30
model.compile(SGD(learning_rate=0.75), loss='sparse_categorical_crossentropy', metrics = ['acc'])
callbacks = [
             EarlyStopping(monitor ='val_acc', patience = 3,mode='max'),
             LearningRateScheduler(scheduler)
             ]
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = batch_size, epochs = epochs, callbacks = callbacks)

Epoch 1/30
528/528 [==============================] - 66s 124ms/step - loss: 3.9615 - acc: 0.5067 - val_loss: 3.4053 - val_acc: 0.5300
Epoch 2/30
528/528 [==============================] - 63s 119ms/step - loss: 3.3456 - acc: 0.5302 - val_loss: 3.2957 - val_acc: 0.5307
Epoch 3/30
528/528 [==============================] - 63s 119ms/step - loss: 3.2784 - acc: 0.5327 - val_loss: 3.2498 - val_acc: 0.5354
Epoch 4/30
528/528 [==============================] - 63s 119ms/step - loss: 3.2068 - acc: 0.5394 - val_loss: 3.1712 - val_acc: 0.5420
Epoch 5/30
528/528 [==============================] - 63s 119ms/step - loss: 3.1478 - acc: 0.5433 - val_loss: 3.1168 - val_acc: 0.5453
Epoch 6/30
528/528 [==============================] - 63s 119ms/step - loss: 3.1136 - acc: 0.5457 - val_loss: 3.1168 - val_acc: 0.5453
Epoch 7/30
528/528 [==============================] - 63s 119ms/step - loss: 3.1136 - acc: 0.5457 - val_loss: 3.1168 - val_acc: 0.5453
Epoch 8/30
528/528 [==============================] - 6

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    25571400    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    5785800     input_2[0][0]                    
_______________________________________________________________________________________

In [ ]:
model.save('/content/drive/My Drive/seq2seq_3조/0.5818model')

INFO:tensorflow:Assets written to: /content/drive/My Drive/seq2seq_3조/0.5818model/assets


In [ ]:
re_model = tf.keras.models.load_model('/content/drive/My Drive/seq2seq_3조/0.5818model')

In [ ]:
re_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    25571400    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    5785800     input_2[0][0]                    
_______________________________________________________________________________________

In [ ]:
re_model.fit([encoder_input_train, decoder_input_train], decoder_target_train, validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),batch_size = 128, epochs = epochs, callbacks = [callbacks])

Epoch 1/30


InvalidArgumentError: ignored

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    25571400    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    5785800     input_2[0][0]                    
_______________________________________________________________________________________

## seq2seq 번역기

In [ ]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
encoder_model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 200)         17047600  
_________________________________________________________________
masking_2 (Masking)          (None, None, 200)         0         
_________________________________________________________________
lstm_2 (LSTM)                [(None, 200), (None, 200) 320800    
Total params: 17,368,400
Trainable params: 17,368,400
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 디코더
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [ ]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

In [ ]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index][::-1]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  그의 항의에 아무런 반응도 없었어요 . 
번역문 : there was no response to his appeal . 
예측문 :  solutions solutions solutions baekseolgi solu


원문 :  쇼맨샤먼이라는 제목으로 배우가 곧 무당이라는 의미를 가지고 있어요 . 
번역문 : its name is showman shaman meaning the actor is a shaman . 
예측문 :  regions debugs debugs debugs leftovers shark compl


원문 :  그리고 내가 가장 좋아하는 고기 반찬들을 먹을수 있어 . 
번역문 : in addition i can eat my favorite dishes made from meat . 
예측문 :  regions solutions solutions baekseolgi solutions baeks


원문 :  너는 첨부 파일을 보기 바래요 . 
번역문 : you should look at the attached file . 
예측문 :  solutions solutions solutions baekseolgi solu


원문 :  나는 매우 유머있고 착실한 소녀이예요 . 
번역문 : i am a really humorous and good girl . 
예측문 :  solutions solutions solutions baekseolgi solu


